## Pre-Processing

* 3 November 2023 Registration open.
* 4 March 2024 Training and development sets available.
* 15 April 2024 Test set available.
* 22 April 2024 Registration closes.
> *** 6 May 2024 Runs submission due to organizers.****
* 20 May 2024 Results notification to participants.
* 3 June 2024 Submission of Working Notes by participants.
* 19 June 2024 Notification of acceptance (peer-reviews).
* 3 July 2024 Camera-ready participant papers due to organizers.
* 9-12 September 2024 EXIST 2024 at CLEF Conference.

In [ ]:
import numpy as np
import pandas as pd
import json
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from transformers import logging, AutoTokenizer, AutoModel,AutoModelForSequenceClassification
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from functools import partial
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
import json
import os
import pandas as pd
import numpy as np
import random
import time
import logging
from datetime import datetime
import sys
from contextlib import redirect_stdout
from io import StringIO
import json
import torch.nn.parallel
# from ai21 import AI21Client
# from ai21.models import ParaphraseStyleType

In [ ]:
df_train = pd.read_json("/kaggle/input/exist-2024/EXIST2024_training.json")
df_train.head()

In [ ]:
df_test = pd.read_json("/kaggle/input/exist-2024/EXIST2023_test_clean.json")
df_test = df_test.T
df_test.head()

In [ ]:
df_train=df_train.T
df_train.head()

In [ ]:
len(df_train)

In [ ]:
df_dev=pd.read_json('/kaggle/input/exist-2024/EXIST2024_dev.json')
df_dev=df_dev.T
df_dev.head()

In [ ]:
df_train["split"].value_counts()

In [ ]:
def assign_labels(dfrow,task="task1"):
    annot_labels=dfrow[f'labels_{task}']
    if annot_labels.count("YES")>=3:
        return "sexist"
    else:
        return "nonsexist"

In [ ]:
def majority_voting_df(df,task="task1"):
    df[f'{task}']=df.apply(assign_labels,axis=1)
    return df

In [ ]:
def preprocess_df(df,task="task1",shuffle=True):
    "Set shuffle as False for test"
#     tasks = ['task1','task2','task3']
    common_cols = ['lang','number_annotators', 'annotators',
       'gender_annotators', 'age_annotators', 'ethnicities_annotators',
       'study_levels_annotators', 'countries_annotators','labels_task1','labels_task2','labels_task3','split']
#     common_cols.extend(["labels_"+col for col in tasks if col!=task])
    df.drop(common_cols,axis=1,inplace = True)
    
    #Renaming task to label
    df = df.rename(columns={task:"label"})
    
    #Shuffling
    ##Fisher-Yates Algorithm
    if shuffle is True:
        num_rows = len(df)
        indices = np.arange(num_rows)

        for i in range(num_rows - 1, 0, -1):
            j = np.random.randint(0, i + 1)
            indices[i], indices[j] = indices[j], indices[i]

        shuffled_df = df.iloc[indices].reset_index(drop=True)
        return shuffled_df
    else:
        return df
    

In [ ]:
def train_val_split(df,splitval):
    split = int(len(df)*splitval)
    return df[:split],df[split:]

In [ ]:
df_train = majority_voting_df(df_train,"task1")
df_train = preprocess_df(df_train)
# df_train,df_val = train_val_split(df_train,0.8)
df_train.head()

In [ ]:
df_dev = majority_voting_df(df_dev,"task1")
df_dev = preprocess_df(df_dev,shuffle=False)
# df_train,df_val = train_val_split(df_train,0.8)
df_dev.head()

In [ ]:
df_test.columns

In [ ]:
df_test.drop(['lang', 'number_annotators', 'annotators',
       'gender_annotators', 'age_annotators', 'ethnicities_annotators',
       'study_levels_annotators', 'countries_annotators', 'split'],axis = 1,inplace = True)

In [ ]:
df_test

In [ ]:
df_train['label'].value_counts()

In [ ]:
# df_val['label'].value_counts()

In [ ]:
LEN_TRAIN,LEN_TEST = len(df_train),len(df_dev)
LEN_TRAIN,LEN_TEST 

## Training

In [ ]:
train_json= df_train.to_json(orient='records') 
label_dict={'nonsexist':0,'sexist':1}

In [ ]:
test_json= df_dev.to_json(orient='records')#ACTUALLY TEST NOT VAL

In [ ]:
class Transformer(nn.Module):

    def __init__(self, base_model, num_classes, method,args):
        super().__init__()
        self.args=args
        self.base_model = base_model
        self.num_classes = num_classes
        self.method = method
        self.hidden_size  = base_model.config.hidden_size
        #self.linear = nn.Linear(base_model.config.hidden_size, num_classes)
        self.linear = nn.Linear(self.hidden_size*2, num_classes)
        self.linear_adjust_dim=nn.Linear(2048,1024)
        self.dropout = nn.Dropout(args.dropout)
        for param in base_model.parameters():
            param.requires_grad_(True)
            
        if torch.cuda.device_count() > 1:
            print("Using", torch.cuda.device_count(), "GPUs!")
            self.base_model = nn.DataParallel(self.base_model)
            
    def word_level_attention(self,lhs):
        qq=lhs.float().to(self.args.device)
        attention_weights=nn.Linear(self.hidden_size,1).to(self.args.device)(qq).tanh()
        attention_weights=attention_weights.squeeze(dim=-1)
        attention_weights=torch.nn.functional.softmax(attention_weights,dim=1)
        attention_weights=torch.unsqueeze(attention_weights,dim=-1)*qq
        word_level_attention=torch.sum(attention_weights,dim=1)
        return word_level_attention
        
    def forward(self, inputs):
        raw_outputs = self.base_model(**inputs)
        hiddens = raw_outputs.last_hidden_state.to(self.args.device)
        cls_feats = hiddens[:, 0, :].to(self.args.device)
        if self.method in ['ce', 'scl']:
            label_feats = None
            word_attention=self.word_level_attention(hiddens)
            concatenated_feats=torch.concat((cls_feats,word_attention),dim=1)
            #predicts = self.linear(self.dropout(cls_feats))
            predicts = self.linear(self.dropout(concatenated_feats))

        else:
            label_feats = hiddens[:, 1:self.num_classes+1, :]
            predicts = torch.einsum('bd,bcd->bc', cls_feats, label_feats)
            word_attention=self.word_level_attention(hiddens)
            concatenated_feats=torch.concat((cls_feats,word_attention),dim=1)
            #print(f"DualCL Dim Concated:Shape is {concatenated_feats.shape}")
            dim_concat_feats=self.linear_adjust_dim(concatenated_feats)
            #print(f"DualCL Dim Adjusted :Shape is {dim_concat_feats.shape}")
            predicts = torch.einsum('bd,bcd->bc', dim_concat_feats, label_feats)
        outputs = {
            'predicts': predicts,
            'cls_feats': cls_feats,
            'hiddens':hiddens,
            'label_feats': label_feats
        }
        return outputs

In [ ]:
class Instructor:

    def __init__(self, args):
        self.args = args
        #self.logger = logger
        #self.logger.info('> creating model {}'.format(args.model_name))
        if args.model_name == 'bert':
            self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
            base_model = AutoModel.from_pretrained('bert-base-uncased')
        elif args.model_name == 'roberta':
            self.tokenizer = AutoTokenizer.from_pretrained('roberta-base', add_prefix_space=True)
            base_model = AutoModel.from_pretrained('roberta-base')
        elif args.model_name == 'bertweet':
            self.tokenizer = tokenizer = AutoTokenizer.from_pretrained("NLP-LTU/bertweet-large-sexism-detector", add_prefix_space=True)
            base_model = AutoModel.from_pretrained("NLP-LTU/bertweet-large-sexism-detector")
        elif args.model_name == 'mbert':
            self.tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
            base_model = AutoModel.from_pretrained("google-bert/bert-base-multilingual-cased")
        elif args.model_name == 't5':
            self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
            base_model = AutoModel.from_pretrained("google/flan-t5-small")
        elif args.model_name=='xlm-r':
            self.tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
            base_model = AutoModel.from_pretrained("FacebookAI/xlm-roberta-base")
        elif args.model_name == 'xlm-r-large':
            self.tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-large")
            base_model = AutoModel.from_pretrained("FacebookAI/xlm-roberta-large")
        else:
            raise ValueError('unknown model')
            
        self.model = Transformer(base_model, args.num_classes, args.method,args)
        self.model.to(args.device)
        if args.device.type == 'cuda':
            print('> cuda memory allocated: {}'.format(torch.cuda.memory_allocated(args.device.index)))
        self._print_args()
        
    def eval_model(self,ytrue,ypred):
        report = classification_report(ytrue,ypred )
        print(report)

        f1= precision_recall_fscore_support( ytrue,ypred , average='binary')
        print("f1 :" , f1)
        accuracy = accuracy_score(ytrue,ypred, )
        print("Accuracy :" , accuracy)

    def _print_args(self):
        print('> training arguments:')
        for arg in vars(self.args):
            print(f">>> {arg}: {getattr(self.args, arg)}")
            
    def max_performance(self,outputs_prob,ytrue):
        complete = int(LEN_TEST / self.args.test_batch_size)
        pred_1 = np.concatenate([t.cpu().numpy() for t in outputs_prob[:complete]])
        pred_2 = outputs_prob[complete].cpu().numpy()

        #print(f"Y true shape:{ytrue.shape}")
        outputs_prob= np.concatenate((pred_1, pred_2))
        #print(f"outputs_prob shape:{outputs_prob.shape}")
        start=time.time()
        m = nn.Softmax(dim=1)
        output_softmax=m(torch.from_numpy(outputs_prob))
        #print(f"Output softmax shape:{output_softmax.shape}")
        y_1=output_softmax[:,1].cpu()#Extracing elements from 1st index
        #print(f"After extracting shape:{y_1.shape}")
        max = 0
        index = 0.1
        for i in range(9000):
            value = 0.1 + i*0.0001
            y_pred1 = np.where(y_1>value, 1, 0)
            f1_macro = precision_recall_fscore_support( ytrue, y_pred1 , average='macro',zero_division=1)
            #acc = accuracy_score( ytrue, y_pred1 )
            if f1_macro[2] > max :
                max = f1_macro[2]
                index = value
        end=time.time()
        print("Max Performance")
        print(f"f1 macro: {max} at threshold {index}")
        print(f"Finished in {end-start} seconds")

    def _train(self, dataloader, criterion, optimizer):
        train_loss, n_correct, n_train = 0, 0, 0
        self.model.train()
        for inputs, targets in tqdm(dataloader, disable=self.args.backend, ascii=' >='):
            inputs = {k: v.to(self.args.device) for k, v in inputs.items()}
            targets = targets.to(self.args.device)
            outputs = self.model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * targets.size(0)
            n_correct += (torch.argmax(outputs['predicts'], -1) == targets).sum().item()
            n_train += targets.size(0)
        return train_loss / n_train, n_correct / n_train

    def _test(self, dataloader, criterion):
        test_loss, n_correct, n_test = 0, 0, 0
        m = nn.Softmax(dim=1)
        self.model.eval()
        l_yout=[]
        l_ytrue=[]
        stored_output=[]
        with torch.no_grad():
            for inputs, targets in tqdm(dataloader, disable=self.args.backend, ascii=' >='):
                inputs = {k: v.to(self.args.device) for k, v in inputs.items()}
                targets = targets.to(self.args.device)
                outputs = self.model(inputs)
                #print("Outputs: ",outputs)
                #print("Predictions: ",outputs['predicts'])
                pred=(torch.argmax(outputs['predicts'],-1)).cpu()
                stored_output.append(outputs['predicts'])
                l_yout.append(pred.numpy())
                #print("ArgMax Pred: ",pred)
                l_ytrue.append(targets.cpu().numpy())
                
                targets = targets.to(self.args.device)
                #print("Targets: ",targets)
                loss = criterion(outputs, targets)
                test_loss += loss.item() * targets.size(0)
                #print(f"Sklearn accuracy is: {skacc}")
                n_correct += (torch.argmax(outputs['predicts'], -1) == targets).sum().item()
                n_test += targets.size(0)
        complete = int(LEN_TEST / self.args.test_batch_size)
        print("LEN TEST: ",LEN_TEST)
        pred_1=np.array(l_yout[:complete]).flatten()
        pred_2=l_yout[complete]
        ypred=np.append(pred_1,pred_2)
        print(f"Samples in Test Set: {len(ypred)}")
        true_1=np.array(l_ytrue[:complete]).flatten()
        true_2=l_ytrue[complete]
        ytrue=np.append(true_1,true_2)
        
        count1_ytrue=np.count_nonzero(ytrue==1)
        count1_ypred=np.count_nonzero(ypred==1)
        print(f"Number of Sexist Examples in Testset: {count1_ytrue}")
        print(f"Number of Sexist Predictions : {count1_ypred}")
        
        self.eval_model(ytrue,ypred)
#         self.max_performance(stored_output,ytrue)
        return test_loss / n_test, n_correct / n_test

    def run(self):
        train_dataloader, test_dataloader = load_data(dataset=self.args.dataset,
                                                      data_dir=self.args.data_dir,
                                                      tokenizer=self.tokenizer,
                                                      train_batch_size=self.args.train_batch_size,
                                                      test_batch_size=self.args.test_batch_size,
                                                      model_name=self.args.model_name,
                                                      method=self.args.method,
                                                      workers=0)
        _params = filter(lambda p: p.requires_grad, self.model.parameters())
        if self.args.method == 'ce':
            criterion = CELoss()
        elif self.args.method == 'scl':
            criterion = SupConLoss(self.args.alpha, self.args.temp,self.args)
        elif self.args.method == 'dualcl':
            criterion = DualLoss(self.args.alpha, self.args.temp,self.args)
        else:
            raise ValueError('unknown method')
        optimizer = torch.optim.AdamW(_params, lr=self.args.lr, weight_decay=self.args.decay)
        best_loss, best_acc = 0, 0
        self.best_model = self.model.state_dict()
        for epoch in range(self.args.num_epoch):
            train_loss, train_acc = self._train(train_dataloader, criterion, optimizer)
            test_loss, test_acc= self._test(test_dataloader, criterion)
            if test_acc > best_acc or (test_acc == best_acc and test_loss < best_loss):
                best_acc, best_loss = test_acc, test_loss
                self.best_model = self.model.state_dict()
                
            
            
            print('{}/{} - {:.2f}%'.format(epoch+1, self.args.num_epoch, 100*(epoch+1)/self.args.num_epoch))
            print('[train] loss: {:.4f}, acc: {:.2f}'.format(train_loss, train_acc*100))
            print('[test] loss: {:.4f}, acc: {:.2f}'.format(test_loss, test_acc*100))
        if self.args.save_model:
                torch.save(self.best_model, self.args.save_path)
                print(f'Model saved to {self.args.save_path}')
        print('best loss: {:.4f}, best acc: {:.2f}'.format(best_loss, best_acc*100))
        return train_loss,train_acc,test_loss,test_acc

In [ ]:
class CELoss(nn.Module):

    def __init__(self):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()

    def forward(self, outputs, targets):
        return self.xent_loss(outputs['predicts'], targets)


class SupConLoss(nn.Module):

    def __init__(self, alpha, temp,args):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()
        self.alpha = alpha
        self.temp = temp
        self.args=args
#         self.hidden_size = 1024 if self.args.model_name=='bertweet' else 768
        if self.args.model_name=='bertweet' or self.args.model_name=='xlm-r-large':
            self.hidden_size = 1024
        elif self.args.model_name=='mbert' or self.args.model_name=='xlm-r':
            self.hidden_size = 768
        elif self.args.model_name=='t5':
            self.hidden_size = 512
        else:
            self.hidden_size = 512
        
        self.linear_adjust_dim=nn.Linear(self.hidden_size*2,self.hidden_size)
        
    def word_level_attention(self,lhs):
        qq=lhs.float().to(self.args.device)
        attention_weights=nn.Linear(self.hidden_size,1).to(self.args.device)(qq).tanh()
        attention_weights=attention_weights.squeeze(dim=-1)
        attention_weights=torch.nn.functional.softmax(attention_weights,dim=1)
        attention_weights=torch.unsqueeze(attention_weights,dim=-1)*qq
        word_level_attention=torch.sum(attention_weights,dim=1)
        return word_level_attention
    
    def nt_xent_loss(self,anchor, target, labels, threshold=0.9):
        '''
        Updated loss function that ignores positive pairs below the threshold
        1. Creates a mask of positives above the threshold
        2. Loss Function, considering only positives above threshold in numerator
           and both positives above threshold and negatives in denominator
        '''
        anchor=anchor.cpu()
        target=target.cpu()
        # Convert cosine similarity array to PyTorch tensor
        #print("Cosine Similarity Matrix: ",cosine_sim)
        # ONE: Create mask of positives above the threshold
        with torch.no_grad():
            cosine_sim = cosine_similarity(anchor, target)
            device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            cosine_sim=torch.tensor(cosine_sim).to(device)
            
            labels = labels.unsqueeze(-1)
            mask_positives = torch.eq(labels, labels.transpose(0, 1)) * (cosine_sim > threshold)
            mask_negatives =  ~torch.eq(labels, labels.transpose(0, 1))

        #print("Mask Positives(above thresh)",mask_positives)
        #print("Mask Negatives",mask_negatives)
        # TWO: Compute log probabilities for positives above threshold
        logits_max, _ = torch.max(cosine_sim / self.temp, dim=1, keepdim=True)
        logits = cosine_sim / self.temp - logits_max.detach()
        exp_logits = torch.exp(logits)
        log_prob = logits - torch.log(exp_logits.sum(dim=1, keepdim=True) + 1e-12)
        #print("All elements log prob: ",log_prob)
        # THREE: Compute loss considering only positives in numerator and both positives and negatives in denominator
        pos_mask_sum = mask_positives.sum(dim=1)
        pos_mask_sum = torch.where(pos_mask_sum == 0, torch.ones_like(pos_mask_sum), pos_mask_sum)
        pos_logits = (mask_positives * log_prob).sum(dim=1) / pos_mask_sum.detach()
        #print("Positives above thresh logits: ",pos_logits)
        neg_mask_sum = mask_negatives.sum(dim=1)
        neg_mask_sum = torch.where(neg_mask_sum == 0, torch.ones_like(neg_mask_sum), neg_mask_sum)
        neg_logits = (mask_negatives * log_prob).sum(dim=1) / neg_mask_sum.detach()
        all_logits = (mask_positives * log_prob).sum(dim=1) + (mask_negatives * log_prob).sum(dim=1)
        #print("Negative logits: ",neg_logits)
        #print("Sum of pos and neg logits: ",all_logits)

        # FOUR: Compute final loss by combining positives (above threshold) and negatives
        alpha = 0.5  # Weighting factor for balancing the importance of positives in numerator and all samples in denominator
        loss = -1 * (alpha * pos_logits + (1 - alpha) * all_logits).mean()
        print("Loss: ",loss)

        return loss

    def forward(self, outputs, targets):
        normed_cls_feats = F.normalize(outputs['cls_feats'], dim=-1)
        ce_loss = (1 - self.alpha) * self.xent_loss(outputs['predicts'], targets)
        self.linear_adjust_dim.to(self.args.device)
        wla=self.word_level_attention(outputs['hiddens'])
        concatenated_feats=torch.concat((normed_cls_feats,wla),dim=1)
        dim_concat_feats=self.linear_adjust_dim(concatenated_feats)
#         print(f"DualCL Dim Concated:Shape is {concatenated_feats.shape}")
#         print(f"DualCL Dim Adjusted :Shape is {dim_concat_feats.shape}")

        #cl_loss = self.alpha * self.nt_xent_loss(normed_cls_feats, normed_cls_feats, targets)
        cl_loss = self.alpha * self.nt_xent_loss(dim_concat_feats,dim_concat_feats, targets)
        return ce_loss + cl_loss

    
class DualLoss(SupConLoss):

    def __init__(self, alpha, temp,args):
        super().__init__(alpha, temp,args)

    def forward(self, outputs, targets):
        normed_cls_feats = F.normalize(outputs['cls_feats'], dim=-1)
        normed_label_feats = F.normalize(outputs['label_feats'], dim=-1)
        normed_lhs_feats=F.normalize(outputs['hiddens'], dim=-1)
        
        self.linear_adjust_dim.to(self.args.device)
        wla=self.word_level_attention(normed_lhs_feats)
        concatenated_feats=torch.concat((normed_cls_feats,wla),dim=1)
        dim_concat_feats=self.linear_adjust_dim(concatenated_feats)
        
        normed_pos_label_feats = torch.gather(normed_label_feats, dim=1, index=targets.reshape(-1, 1, 1).expand(-1, 1, normed_label_feats.size(-1))).squeeze(1)
        ce_loss = (1 - self.alpha) * self.xent_loss(outputs['predicts'], targets)
#         cl_loss_1 = 0.5 * self.alpha * self.nt_xent_loss(normed_pos_label_feats, normed_cls_feats, targets)
#         cl_loss_2 = 0.5 * self.alpha * self.nt_xent_loss(normed_cls_feats, normed_pos_label_feats, targets)
        #cl_loss_1 = 0.5 * self.alpha * self.nt_xent_loss(normed_pos_label_feats, dim_concat_feats, targets)
        cl_loss_1 = 0.5 * self.alpha * self.nt_xent_loss(wla,wla, targets)
        cl_loss_2= 0.5 * self.alpha * self.nt_xent_loss(dim_concat_feats, dim_concat_feats, targets)
        return ce_loss + cl_loss_1 + cl_loss_2

In [ ]:
class MyDataset(Dataset):
    
    def __init__(self, raw_data, label_dict,tokenizer, model_name, method):
        label_list = list(label_dict.keys()) if method not in ['ce', 'scl'] else []
        sep_token = ['[SEP]'] if model_name == 'bertweet' else ['</s>']
        dataset = list()
        for data in raw_data:
            tokens = data['tweet'].lower().split(' ')
            label_id = label_dict[data['label']]
            dataset.append((label_list + sep_token + tokens, label_id))
        self._dataset = dataset

    def __getitem__(self, index):
        return self._dataset[index]

    def __len__(self):
        return len(self._dataset)


def my_collate(batch, tokenizer, method, num_classes):
    tokens, label_ids = map(list, zip(*batch))
    text_ids = tokenizer(tokens,
                         padding=True,
                         truncation=True,
                         max_length=128,
                         is_split_into_words=True,
                         add_special_tokens=True,
                         return_tensors='pt')
    if method not in ['ce', 'scl']:
        positions = torch.zeros_like(text_ids['input_ids'])
        positions[:, num_classes:] = torch.arange(0, text_ids['input_ids'].size(1)-num_classes)
        text_ids['position_ids'] = positions
    return text_ids, torch.tensor(label_ids)

def load_data(dataset, data_dir, tokenizer, train_batch_size, test_batch_size, model_name, method, workers):
    if dataset=='exist':
        train_data = json.loads(train_json)
        test_data = json.loads(test_json)
        label_dict={'nonsexist':0,'sexist':1} 
    else:
        raise ValueError('unknown dataset')
    trainset = MyDataset(train_data, label_dict, tokenizer, model_name, method)
    testset = MyDataset(test_data, label_dict, tokenizer, model_name, method)
    collate_fn = partial(my_collate, tokenizer=tokenizer, method=method, num_classes=len(label_dict))
    train_dataloader = DataLoader(trainset, train_batch_size, shuffle=True, num_workers=workers, collate_fn=collate_fn, pin_memory=True)
    test_dataloader = DataLoader(testset, test_batch_size, shuffle=False, num_workers=workers, collate_fn=collate_fn, pin_memory=True)
    return train_dataloader, test_dataloader

In [ ]:
class Args:
    def __init__(self):
        self.num_classes = 2
        self.data_dir = 'data'
        self.dataset = 'exist'  
        """
        berTweet->1024 size vector
        mBert->768 size vector
        """
        self.model_name = 'xlm-r'     #'bertweet'  
        self.method = 'scl'  
        self.train_batch_size = 16
        self.test_batch_size = 16
        self.num_epoch = 30
        self.lr = 1.0169734125025231e-06
        self.decay = 0.0025948655072046897
        self.dropout=0.36167949934204113
        self.alpha = 0.5
        self.temp =  0.09770519113292521
        self.backend = False
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        #self.device=accelerator.device
        self.save_model = True
        self.save_path = 'best_model.pth'
        
        

args = Args()

In [ ]:
# import sys

# instructor = Instructor(args)
# instructor.run()

In [ ]:
# unhardcode max perf
# remove train 100  
# add code to save best weights on val
"""TODO STILL
run these on final test at the very very end
address to cpu thingy to make it efficient
run inference on test set in another notebook
"""

In [ ]:
captured_output = StringIO()

with redirect_stdout(captured_output):
    instructor = Instructor(args)
    instructor.run()

captured_text = captured_output.getvalue()

with open('Task1_output.txt', 'w+') as f:
    f.write(captured_text)
    

In [ ]:
file = open("Task1_output.txt","r")
text=file.readlines()
len(text)

processed_output=[]
for line in text:
    if line.find("Loss:  tensor(")==-1:
        processed_output.append(line)
        
with open("Task2_processed_output.txt","w+") as file:
    file.writelines(processed_output)

## Using Saved Model for Predictions

In [ ]:
# input_text = "I hate women"
# inputs = tokenizer(input_text, return_tensors='pt').to(args.device)

# with torch.no_grad():
#     outputs = model(inputs)
#     predictions = torch.argmax(outputs['predicts'], dim=1).cpu().numpy()

# label_dict = {0: 'nonsexist', 1: 'sexist'}
# predicted_label = label_dict[predictions[0]]
# print(f"Predicted label: {predicted_label}")

In [ ]:

# predictions_list = []

# for index, row in df_val[:10].iterrows():
#     print(index)
#     row_id = row['id_EXIST']
#     text = row['tweet']
    
#     prediction = get_predictions(model,text,label_dict)
    
#     prediction_dict = {
#         'id': row_id,
#         'value': prediction,
#         'test_case':"EXIST2024"
#     }
    
#     predictions_list.append(prediction_dict)

# with open('predictions.json', 'w') as json_file:
#     json.dump(predictions_list, json_file)

In [ ]:
import json

In [ ]:
def get_predictions(model,text,label_dict={0: 'nonsexist', 1: 'sexist'}):
    inputs = tokenizer(text, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(inputs)
        predictions = torch.argmax(outputs['predicts'], dim=1).cpu().numpy()

    predicted_label = label_dict[predictions[0]]
    return predicted_label
    

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("NLP-LTU/bertweet-large-sexism-detector", add_prefix_space=True)
# base_model = AutoModel.from_pretrained("NLP-LTU/bertweet-large-sexism-detector")
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
base_model = AutoModel.from_pretrained("FacebookAI/xlm-roberta-base")
model = Transformer(base_model, args.num_classes, args.method, args).to(args.device)
model.load_state_dict(torch.load(args.save_path))
label_dict={0: 'NO', 1: 'YES'}
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def json_predictions(df,fname):
    predictions_list = []

    for index, row in df.iterrows():
        row_id = row['id_EXIST']
        text = row['tweet']

        prediction = get_predictions(model,text,label_dict)

        prediction_dict = {
            'id': row_id,
            'value': [prediction],
            'test_case':"EXIST2024"
        }

        predictions_list.append(prediction_dict)

    with open(fname, 'w') as json_file:
        json.dump(predictions_list, json_file)
    

In [ ]:
# fname="team_aditya_exist2024_task1.json"
# json_predictions(df_dev,fname)

In [ ]:
# with open(fname,"r") as file:
#     text1=json.load(file)

In [ ]:
# text1

## Eval

In [ ]:
# !pip install -q PyEvALL==0.1.63

In [ ]:
# from pyevall.evaluation import PyEvALLEvaluation
# from pyevall.utils.utils import PyEvALLUtils

In [ ]:
# # predictions = "test/hard/EXIST2024_test_task1_baseline_2.json"
# # gold = "test/hard/EXIST2024_test_task1_gold_hard.json" 
# gold = "/kaggle/input/exist-2024/EXIST2024_dev_task1_gold_hard.json"
# predictions = fname
# test = PyEvALLEvaluation()
# params= dict() 
# params[PyEvALLUtils.PARAM_REPORT]= PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED 
# metrics=["ICM", "ICMNorm" ,"FMeasure"] 
# report= test.evaluate(predictions, gold, metrics, **params) 
# report.print_report()

## Baseline Eval

In [ ]:
def json_predictions_baseline(df):
    predictions_list_major = []
    predictions_list_minor = []

    for index, row in df.iterrows():
        row_id = row['id_EXIST']
        text = row['tweet']

        prediction = get_predictions(model,text,label_dict)

        prediction_dict = {
            'test_case':"EXIST2024",
            'id': row_id,
            'value': prediction,
        }
        if prediction=='NO':
            predictions_list_major.append(prediction_dict)
        else:
            predictions_list_minor.append(prediction_dict)
        
#         predictions_list.append(prediction_dict)

    with open('baseline_major', 'w') as json_file:
        json.dump(predictions_list_major, json_file)
        
    with open("baseline_minor", 'w') as json_file:
        json.dump(predictions_list_minor, json_file)
    

In [ ]:
# # fname="team_aditya_exist2024_task1.json"
# json_predictions_baseline(df_dev)

In [ ]:
# gold_major="/kaggle/input/exist-2024/EXIST2024_dev_task1_majority_class_hard.json"
# gold_minor="/kaggle/input/exist-2024/EXIST2024_dev_task1_minority_class_hard.json"

In [ ]:
# # predictions = "test/hard/EXIST2024_test_task1_baseline_2.json"
# # gold = "test/hard/EXIST2024_test_task1_gold_hard.json" 
# predictions_major="baseline_major"
# predictions_minor="baseline_minor"
# test = PyEvALLEvaluation()
# params= dict() 
# params[PyEvALLUtils.PARAM_REPORT]= PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED 
# # metrics=["ICM", "ICMNorm" ,"FMeasure"] 
# metrics=["FMeasure"] 
# print("*****MAJOR*****")
# report_major= test.evaluate(predictions_major, gold_major, metrics, **params) 
# report_major.print_report()
# print("*****MINOR*****")
# report_minor= test.evaluate(predictions_minor, gold_minor, metrics, **params) 
# report_minor.print_report()

# BASELINE SCORES-:
# MAJORITY:0.6215139442231076
# MINORITY:0.7089552238805971

In [ ]:
# with open("/kaggle/input/exist-2024/EXIST2024_dev_task1_majority_class_hard.json","r") as file:
#     text1=json.load(file)
# text1



##  FINAL PREDICTIONS ON TEST SET

In [ ]:
fname="task1_hard_ADITYA_1.json"
json_predictions(df_test,fname)